In [26]:
import pandas as pd, numpy as np
import cv2,PIL,pyvips,glob,os,pickle,random
import matplotlib.pyplot as plt
import skimage.io as sk
from tqdm import tqdm
from matplotlib.patches import Rectangle
import collections
RANDOM_STATE = 41
def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
fix_seed(RANDOM_STATE)

In [27]:
MULS = {i:2**i for i in range(11)}
MULS

{0: 1, 1: 2, 2: 4, 3: 8, 4: 16, 5: 32, 6: 64, 7: 128, 8: 256, 9: 512, 10: 1024}

In [28]:
PAGE = 3

In [29]:
# DATA_DIR = '/home/jovyan/data'
DATA_DIR = os.environ['DIR_RAW_DATA']
RAW_IMG_DIR = f'{DATA_DIR}/images'

dir_images = f'{DATA_DIR}/images/'
df = pd.read_csv(f'{DATA_DIR}/train.csv')
df['path'] = dir_images +df.filename
df['slideid'] = df.filename.str.split('.').str[0]
df['og_bbox'] = df.apply(lambda x: [x.x1,x.y1,x.x2,x.y2] ,axis=1)
df['corr_bbox'] = df.apply(lambda x: [x.x1,x.y1,min(x.max_x,x.x2),min(x.max_y,x.y2)] ,axis=1)

df['w'] = df.x2-df.x1
df['h'] = df.y2-df.y1
df['xpad'] = df.max_x - df.x2
df['ypad'] = df.max_y - df.y2

df

,filename,x1,x2,y1,y2,max_x,max_y,path,slideid,og_bbox,corr_bbox,w,h,xpad,ypad
0,bGaslniO4a_a.tif,29348,30108,28404,29675,82944,197632,/home/jovyan/data/images/bGaslniO4a_a.tif,bGaslniO4a_a,"[29348, 28404, 30108, 29675]","[29348, 28404, 30108, 29675]",760,1271,52836,167957
1,bGaslniO4a_a.tif,11735,12379,70274,71195,82944,197632,/home/jovyan/data/images/bGaslniO4a_a.tif,bGaslniO4a_a,"[11735, 70274, 12379, 71195]","[11735, 70274, 12379, 71195]",644,921,70565,126437
2,2qj5MlLLBT_a.tif,11185,12276,11571,12671,82944,196608,/home/jovyan/data/images/2qj5MlLLBT_a.tif,2qj5MlLLBT_a,"[11185, 11571, 12276, 12671]","[11185, 11571, 12276, 12671]",1091,1100,70668,183937
3,2qj5MlLLBT_a.tif,14380,15583,11252,12434,82944,196608,/home/jovyan/data/images/2qj5MlLLBT_a.tif,2qj5MlLLBT_a,"[14380, 11252, 15583, 12434]","[14380, 11252, 15583, 12434]",1203,1182,67361,184174
4,2qj5MlLLBT_a.tif,12162,13834,71136,72440,82944,196608,/home/jovyan/data/images/2qj5MlLLBT_a.tif,2qj5MlLLBT_a,"[12162, 71136, 13834, 72440]","[12162, 71136, 13834, 72440]",1672,1304,69110,124168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,9YyMyBInqO_b.tif,65321,65970,34804,35426,80128,197888,/home/jovyan/data/images/9YyMyBInqO_b.tif,9YyMyBInqO_b,"[65321, 34804, 65970, 35426]","[65321, 34804, 65970, 35426]",649,622,14158,162462
923,9YyMyBInqO_b.tif,66034,66727,34958,35471,80128,197888,/home/jovyan/data/images/9YyMyBInqO_b.tif,9YyMyBInqO_b,"[66034, 34958, 66727, 35471]","[66034, 34958, 66727, 35471]",693,513,13401,162417
924,4q4lgKXbiH_a.tif,66655,67388,28633,29201,83200,197888,/home/jovyan/data/images/4q4lgKXbiH_a.tif,4q4lgKXbiH_a,"[66655, 28633, 67388, 29201]","[66655, 28633, 67388, 29201]",733,568,15812,168687
925,4q4lgKXbiH_b.tif,14925,15734,33552,34465,83200,197888,/home/jovyan/data/images/4q4lgKXbiH_b.tif,4q4lgKXbiH_b,"[14925, 33552, 15734, 34465]","[14925, 33552, 15734, 34465]",809,913,67466,163423


In [30]:
df[df.og_bbox!=df.corr_bbox]

,filename,x1,x2,y1,y2,max_x,max_y,path,slideid,og_bbox,corr_bbox,w,h,xpad,ypad
539,G97Fh5Kl2L_b.tif,79207,83103,74360,79186,82944,197632,/home/jovyan/data/images/G97Fh5Kl2L_b.tif,G97Fh5Kl2L_b,"[79207, 74360, 83103, 79186]","[79207, 74360, 82944, 79186]",3896,4826,-159,118446


In [31]:
df.describe()

,x1,x2,y1,y2,max_x,max_y,w,h,xpad,ypad
count,927.00000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000
mean,41948.78425,43057.828479,80278.612729,81342.675297,82675.572816,196794.407767,1109.044229,1064.062567,39617.744337,115451.732470
std,24962.85062,24972.369250,52738.948864,52749.569806,692.237246,3401.840356,726.683765,645.719471,24959.371377,52486.487537
min,86.00000,1526.000000,2783.000000,3902.000000,73728.000000,173056.000000,13.000000,11.000000,-159.000000,512.000000
25%,18055.00000,19111.000000,31372.500000,32284.000000,82688.000000,197632.000000,707.500000,710.000000,15767.000000,73429.500000
50%,39238.00000,40193.000000,73238.000000,74241.000000,82688.000000,197888.000000,943.000000,915.000000,42346.000000,122578.000000
75%,65878.00000,66801.500000,122656.500000,124182.000000,82944.000000,198144.000000,1277.500000,1245.500000,63701.000000,164340.000000
max,82331.00000,83103.000000,196540.000000,197376.000000,83200.000000,198656.000000,9576.000000,7479.000000,81418.000000,193160.000000


In [32]:
df.filename.nunique()

251

In [33]:
bads = ['7HxL729fl6_b_38839_78455_39604_79279', 'HpWI7vJms2_a_71182_57113_71902_57462', 'i9xm71KbYG_b_31467_113976_31986_114541', 'JvxiXClFKl_a_20316_9218_20451_9502', 'JvxiXClFKl_a_69751_37180_70029_37225', 'Lzx7XfUujk_a_13308_34726_13336_34737', 'Lzx7XfUujk_b_10468_84122_10792_84991', 'Lzx7XfUujk_b_15486_145827_15738_146587', 'Lzx7XfUujk_b_17181_145345_17593_145910', 'Lzx7XfUujk_b_53455_82217_54007_82559', 'Lzx7XfUujk_b_63207_142583_63569_143562', 'Lzx7XfUujk_b_64715_142167_65022_142754', 'Lzx7XfUujk_b_67782_116477_68140_117554', 'Lzx7XfUujk_b_8802_84908_9241_85324', 'rzsagNFXMn_a_15458_148084_15486_148204', 'TFrBjcO8nJ_b_63103_127967_63818_128712', 'WipCgQtJPE_b_25648_118162_26120_118743', 'yJxYpOCh6m_b_18928_24013_18941_24038']
df['uniqid'] = df.slideid + '_'+ df.og_bbox.apply(lambda x: '_'.join([str(i) for i in x]))
df_bads = df[df.uniqid.isin(bads)].reset_index(drop=True)
df = df[~df.uniqid.isin(bads)].reset_index(drop=True)
print(f'dropped {len(df_bads)}',len(df))

dropped 18 909


In [34]:
df.describe(percentiles=[i/10 for i in range(10)])

,x1,x2,y1,y2,max_x,max_y,w,h,xpad,ypad
count,909.000000,909.000000,909.000000,909.000000,909.000000,909.000000,909.000000,909.000000,909.000000,909.000000
mean,42043.374037,43167.031903,80006.308031,81081.381738,82731.370737,196806.266227,1123.657866,1075.073707,39564.338834,115724.884488
std,24986.106447,24990.765606,52846.122509,52856.565687,432.854403,3424.380257,725.616618,645.684544,24991.276035,52582.683703
min,86.000000,1526.000000,2783.000000,3902.000000,78848.000000,173056.000000,187.000000,138.000000,-159.000000,512.000000
0%,86.000000,1526.000000,2783.000000,3902.000000,78848.000000,173056.000000,187.000000,138.000000,-159.000000,512.000000
10%,10522.000000,11545.400000,17764.600000,18831.000000,82688.000000,195072.000000,533.800000,531.600000,8250.200000,38487.400000
20%,15613.800000,16745.000000,25987.000000,27127.000000,82688.000000,197632.000000,667.600000,662.000000,13959.000000,65616.400000
30%,21026.000000,22001.600000,36076.400000,37025.800000,82688.000000,197632.000000,770.400000,758.400000,18142.600000,83737.600000
40%,27652.400000,28836.400000,61874.600000,62880.800000,82688.000000,197888.000000,854.400000,833.000000,23867.600000,111865.800000
50%,39568.000000,40598.000000,72852.000000,74011.000000,82688.000000,197888.000000,951.000000,923.000000,42167.000000,122854.000000


In [35]:
s=500
df[(df.w<s) | (df.h<s)].shape

(109, 16)

In [36]:
m=MULS[PAGE]
d=df[['w','h']]/m
d.describe(percentiles=[i/10 for i in range(10)])

,w,h
count,909.000000,909.000000
mean,140.457233,134.384213
std,90.702077,80.710568
min,23.375000,17.250000
0%,23.375000,17.250000
10%,66.725000,66.450000
20%,83.450000,82.750000
30%,96.300000,94.800000
40%,106.800000,104.125000
50%,118.875000,115.375000


In [37]:
d[d.w>512]

,w,h
62,662.750,688.250
70,600.000,365.500
71,574.750,353.250
72,562.625,462.000
523,1197.000,768.125
524,560.125,646.250
526,741.625,934.875
751,517.500,306.000


In [38]:
df[df.slideid=='pfgu5Dsc2o_a']

,filename,x1,x2,y1,y2,max_x,max_y,path,slideid,og_bbox,corr_bbox,w,h,xpad,ypad,uniqid
192,pfgu5Dsc2o_a.tif,16958,17234,24755,24971,82944,185088,/home/jovyan/data/images/pfgu5Dsc2o_a.tif,pfgu5Dsc2o_a,"[16958, 24755, 17234, 24971]","[16958, 24755, 17234, 24971]",276,216,65710,160117,pfgu5Dsc2o_a_16958_24755_17234_24971
193,pfgu5Dsc2o_a.tif,65178,65500,22160,22440,82944,185088,/home/jovyan/data/images/pfgu5Dsc2o_a.tif,pfgu5Dsc2o_a,"[65178, 22160, 65500, 22440]","[65178, 22160, 65500, 22440]",322,280,17444,162648,pfgu5Dsc2o_a_65178_22160_65500_22440
194,pfgu5Dsc2o_a.tif,65905,66280,61838,62071,82944,185088,/home/jovyan/data/images/pfgu5Dsc2o_a.tif,pfgu5Dsc2o_a,"[65905, 61838, 66280, 62071]","[65905, 61838, 66280, 62071]",375,233,16664,123017,pfgu5Dsc2o_a_65905_61838_66280_62071
195,pfgu5Dsc2o_a.tif,67975,68395,60749,60887,82944,185088,/home/jovyan/data/images/pfgu5Dsc2o_a.tif,pfgu5Dsc2o_a,"[67975, 60749, 68395, 60887]","[67975, 60749, 68395, 60887]",420,138,14549,124201,pfgu5Dsc2o_a_67975_60749_68395_60887


In [45]:
def get_bboxes_in_region(d,slideid,fname,slide,X1,Y1,X2,Y2,reg_pad,H,W, vis=False):
    reg_annotations = []
    for crop_id,row in d.iterrows():
        og_bbox = np.array(row['og_bbox']).astype(np.int32)
        bbox = (og_bbox/m).astype(np.int32)
        x1,y1,x2,y2 = bbox

        if x2>slide.width:
            x2 = slide.width
        if y2>slide.height:
            y2=slide.height

        # if (X1<=x1<=X2 or X1<=x2<=X2) and (Y1<=y1<=Y2 or  Y1<=y2<=Y2):
        if x1 < X2 and x2 > X1 and y1 < Y2 and y2 > Y1:
            b_x1_abs = max(X1,x1)
            b_y1_abs = max(Y1,y1)
            b_x2_abs = min(X2,x2)
            b_y2_abs = min(y2,Y2)
            b_w = b_x2_abs - b_x1_abs
            b_h = b_y2_abs - b_y1_abs
            #print(X1,Y1,X2,Y2, x1,y1,x2,y2,b_w,b_h)
            b_x1_rel = max(0, b_x1_abs - X1)
            b_y1_rel = max(0, b_y1_abs - Y1)
            b_x2_rel = b_x1_rel + b_w
            b_y2_rel = b_y1_rel + b_h
            crop_bbox = [b_x1_rel,b_y1_rel,b_x2_rel,b_y2_rel]
            full_bbox = [x1,y1,x2,y2]
            b_w = b_x2_rel - b_x1_rel
            b_h = b_y2_rel - b_y1_rel
            reg_annotations.append([slideid,fname,og_bbox,full_bbox,crop_bbox, b_h, b_w,H,W])
            if vis:
                reg_pad = cv2.rectangle(reg_pad,crop_bbox[:2],crop_bbox[2:],255,1)

    assert len(reg_annotations) > 0
    return reg_annotations, reg_pad

In [48]:
PAD_MIN = 100; PAD_MAX = 500

fix_seed(RANDOM_STATE)
out_dir = f'p{PAGE}_crops_p_{PAD_MIN}_{PAD_MAX}'
out_dir = os.path.join(os.environ['DIR_PROCESSED_DATA'],out_dir)
os.makedirs(out_dir,exist_ok=True)
cols = ['x1','y1','x2','y2']
m = MULS[PAGE]

ANNOTATIONS = []
for filename,d in tqdm(df.groupby('filename')):
    # pass
    d = d.reset_index(drop=True)
    slideid = filename.replace('.tif','')
    path = d.path.values[0]
    slide = pyvips.Image.new_from_file(path,page=PAGE)

    for crop_id,row in d.iterrows():
        #og_bbox = row[cols].values.astype(np.int32)
        og_bbox = np.array(row['og_bbox']).astype(np.int32)
        bbox = (og_bbox/m).astype(np.int32)
        x1,y1,x2,y2 = bbox
        if x2>slide.width:
            x2 = slide.width
        if y2>slide.height:
            y2=slide.height
 
        padx1 = random.randint(PAD_MIN,PAD_MAX)
        pady1 = random.randint(PAD_MIN,PAD_MAX)
        padx2 = random.randint(PAD_MIN,PAD_MAX)
        pady2 = random.randint(PAD_MIN,PAD_MAX)
        X1 = x1-padx1
        Y1 = y1-pady1
        X2 = x2+padx2
        Y2 = y2+pady2
        X1 = max(X1,0)
        Y1 = max(Y1,0)
        X2 = min(X2,slide.width)
        Y2 = min(Y2,slide.height)
        w = x2-x1; h = y2-y1
        W = X2-X1; H = Y2-Y1


        reg_pad = pyvips.Region.new(slide).fetch(X1,Y1,W,H)
        reg_pad = np.ndarray(buffer=reg_pad,dtype=np.uint8,shape=(H, W, 3))

        fname = f'{slideid}_{crop_id}.png'
        reg_annotations,reg_pad = get_bboxes_in_region(d,slideid,fname,slide,X1,Y1,X2,Y2,reg_pad,H,W,vis=False)
       

        ANNOTATIONS += reg_annotations
        reg_pad = PIL.Image.fromarray(reg_pad)
        reg_pad.save(f'{out_dir}/{fname}',quality=90)
       

100%|██████████| 247/247 [01:26<00:00,  2.85it/s]


In [50]:
d = pd.DataFrame(ANNOTATIONS,columns=['slide_id','file_id','og_bbox','full_bbox','crop_bbox','bb_h','bb_w','height','width'])
d.to_pickle(f'{out_dir}/anno.pkl')
d.describe(percentiles=[i/10 for i in range(10)])

,bb_h,bb_w,height,width
count,1238.000000,1238.000000,1238.000000,1238.000000
mean,130.408724,135.981422,735.121163,733.323910
std,76.310566,85.864205,183.509812,183.416893
min,1.000000,4.000000,275.000000,198.000000
0%,1.000000,4.000000,275.000000,198.000000
10%,63.000000,62.700000,492.000000,500.000000
20%,80.000000,81.000000,578.400000,581.400000
30%,93.000000,95.000000,643.000000,639.000000
40%,102.000000,104.000000,685.000000,680.000000
50%,114.000000,117.000000,730.000000,723.000000
